In [45]:
import numpy as np
import pandas as pd
from codebase.file_utils import save_obj, load_obj
import os


In [56]:
def form_df(samples, num_chains, rows):
    dfs = []
    for cn in range(num_chains):
        for r in range(rows):
            if rows>1:
                df = pd.DataFrame(
                        samples[:,cn, r]
                )
            else:
                df = pd.DataFrame(
                        samples[:,cn]
                )
            df.insert(0, 'idx', np.arange(df.shape[0]))
            df = df.melt(id_vars ='idx', var_name = 'col')
            df.insert(1, 'row' , r)
            df.insert(1, 'chain', cn)
            dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)


def get_post_df(samples):
    num_chains = samples.shape[1]
    if samples.ndim > 3:
        rows = samples.shape[2]
        df = form_df(samples, num_chains, rows)
    else:
        rows = 1
        df = form_df(samples, num_chains, 1)
    return df

def write_csv(log_dir, filename):
    ps = load_obj('ps', log_dir)
    dfs=[]
    for name in ps.keys():
        if name not in ['zz', 'yy']:
            df = get_post_df(ps[name])
            df['name'] = name
            dfs.append(df)

    outdf = pd.concat(dfs)
    outdf.to_csv("log/FND/Kostas_files/%s.csv"%filename, index=False)

In [57]:
directory = 'log/FND/PPP_rename/'
for filename in os.listdir(directory):
    if not filename.endswith("Store"): 
        log_dir = os.path.join(directory, filename)+'/'
#         print(log_dir, filename)
        write_csv(log_dir, filename)

In [27]:
outdf

,idx,chain,row,col,value,name
0,0,0,0,0,-1.211720,alpha
1,1,0,0,0,-1.140967,alpha
2,2,0,0,0,-1.155425,alpha
3,3,0,0,0,-1.342274,alpha
4,4,0,0,0,-1.498333,alpha
...,...,...,...,...,...,...
287995,1995,3,5,5,0.635970,Omega_cov
287996,1996,3,5,5,0.268265,Omega_cov
287997,1997,3,5,5,0.128561,Omega_cov
287998,1998,3,5,5,0.962453,Omega_cov
